In [1]:

%run ../py/scraping_utils.py
from IPython.display import clear_output

su = ScrapingUtilities()
if not su.s.pickle_exists('connection_info_set'):
    
    # Spoof a browser and scrape it off the web
    driver = su.get_driver('FireFox')
    
    # Go to MineCraft Server List website
    page_url = 'https://minecraft-server-list.com/'
    su.driver_get_url(driver, page_url)
    
    # Get server page URLs
    connection_info_set = set()
    server_list_xpath = '/html/body/div[2]/div/section[3]/div[2]/div[2]/table/tbody'
    connection_info_xpath = '/html/body/div[3]/div/section/div[1]/span/h1'
    web_element = su.get_web_element(driver, server_list_xpath)
    FirefoxWebElements_list = web_element.find_elements_by_tag_name('a')
    page_urls_list = [FirefoxWebElement_obj.get_attribute('href') for FirefoxWebElement_obj in FirefoxWebElements_list]
    
    # Get connection info list
    for page_url in page_urls_list:

        # Get server page
        clear_output(wait=True)
        su.driver_get_url(driver, page_url)

        # Get connection info
        web_element = su.get_web_element(driver, connection_info_xpath)
        FirefoxWebElements_list = web_element.find_elements_by_tag_name('a')
        connection_info = FirefoxWebElements_list[-1].text
        connection_info_set.add(connection_info)
    
    driver.close()
    
    su.s.store_objects(connection_info_set=connection_info_set)
else:
    connection_info_set = su.s.load_object('connection_info_set')

In [8]:

%run ../load_magic/lists.py
from mcstatus import MinecraftServer
import socket

for host_str in connection_info_set:
    print()
    java_server_obj = MinecraftServer(host_str)
    try:
        status = java_server_obj.status()
        print(f'The {host_str} server has {status.players.online} players and replied in {status.latency} ms')
    except socket.error as e:
        print(f'*** {host_str} status error: {str(e).strip()} ***')
    try:
        latency = java_server_obj.ping()
        print(f'The {host_str} server replied in {latency} ms')
    except socket.error as e:
        print(f'*** {host_str} ping error: {str(e).strip()} ***')
    try:
        query = java_server_obj.query()
        print(f'The {host_str} server has the following players online: {conjunctify_nouns(query.players.names)}')
    except socket.error as e:
        print(f'*** {host_str} query error: {str(e).strip()} ***')


The mc.insanitycraft.net server has 37 players and replied in 93.936 ms
*** mc.insanitycraft.net ping error: Server did not respond with any information! ***
*** mc.insanitycraft.net query error: timed out ***

The play.skyblock.net server has 146 players and replied in 22.326 ms
*** play.skyblock.net ping error: Server did not respond with any information! ***
*** play.skyblock.net query error: timed out ***

*** play.applecraft.org status error: [Errno 11001] getaddrinfo failed ***
*** play.applecraft.org ping error: [Errno 11001] getaddrinfo failed ***
*** play.applecraft.org query error: [Errno 11001] getaddrinfo failed ***

The pixel.mc-complex.com server has 929 players and replied in 25.766 ms
*** pixel.mc-complex.com ping error: Server did not respond with any information! ***
*** pixel.mc-complex.com query error: timed out ***

The two.oneblockmc.com server has 454 players and replied in 22.351 ms
*** two.oneblockmc.com ping error: timed out ***
*** two.oneblockmc.com query e

In [12]:

host_str = 'play.dogecraft.net'
java_server_obj = MinecraftServer(host_str)
status = java_server_obj.status()
Version_obj = status.version
# Version_obj.protocol, Version_obj.name
Players_obj = status.players
Players_obj.online, Players_obj.max, Players_obj.sample

(28, 100, None)


----

In [1]:

# If you know the host and port, you may skip this and use MinecraftServer('example.org', 1234)
server = MinecraftServer.lookup('play.ecc.eco:25565')

In [2]:

# 'status' is supported by all Minecraft servers that are version 1.7 or higher.
status = server.status()
print('The server has {0} players and replied in {1} ms'.format(status.players.online, status.latency))

The server has 117 players and replied in 30.92 ms


In [3]:

# 'ping' is supported by all Minecraft servers that are version 1.7 or higher.
# It is included in a 'status' call, but is exposed separate if you do not require the additional info.
latency = server.ping()
print('The server replied in {0} ms'.format(latency))

timeout: timed out

In [4]:

# 'query' has to be enabled in a servers' server.properties file.
# It may give more information than a ping, such as a full player list or mod information.
query = server.query()
print('The server has the following players online: {0}'.format(', '.join(query.players.names)))

timeout: timed out

In [5]:

print(['server.{}'.format(fn) for fn in dir(server) if not fn.startswith('_')])

['server.async_ping', 'server.async_query', 'server.async_status', 'server.host', 'server.lookup', 'server.ping', 'server.port', 'server.query', 'server.status']


In [7]:

print(['status.{}'.format(fn) for fn in dir(status) if not fn.startswith('_')])

['status.Players', 'status.Version', 'status.description', 'status.favicon', 'status.latency', 'status.players', 'status.raw', 'status.version']


In [20]:

Version_obj = status.version
print(['Version_obj.{}'.format(fn) for fn in dir(Version_obj) if not fn.startswith('_')])

['Version_obj.name', 'Version_obj.protocol']


In [24]:

Version_obj.protocol

47

In [22]:

Version_obj.name

'Waterfall 1.17.1'

In [17]:

Players_obj = status.players
print(['Players_obj.{}'.format(fn) for fn in dir(Players_obj) if not fn.startswith('_')])

['Players_obj.Player', 'Players_obj.max', 'Players_obj.online', 'Players_obj.sample']


In [28]:

Players_obj.online

117

In [27]:

Players_obj.max

2000

In [31]:

Player_obj = Players_obj.sample[0]
print(['Player_obj.{}'.format(fn) for fn in dir(Player_obj) if not fn.startswith('_')])

['Player_obj.id', 'Player_obj.name']


In [32]:

Player_obj.id

'00000000-0000-0000-0000-000000000000'

In [33]:

Player_obj.name

'§aJoin now and play together with §6NCTBoys§a ...'


----

In [35]:

%run ../load_magic/soup.py
from urllib.error import HTTPError

try:
    page_url = 'https://minecraft-server-list.com/server/7038/'
    page_soup = get_page_soup(page_url)
except HTTPError as e:
    print(str(e).strip())

HTTP Error 403: Forbidden


In [11]:

import random

FirefoxWebElement_obj = random.choice(FirefoxWebElements_list)
[f'FirefoxWebElement_obj.{fn}' for fn in dir(FirefoxWebElement_obj) if not fn.startswith('_')]
FirefoxWebElement_obj.get_attribute('outerHTML')

'<a href="//minecraft-server-list.com/server/461166/" title="One Block MC Minecraft Server">One Block MC</a>'

In [2]:

from urllib.error import HTTPError

try:
    page_soup = get_page_soup(page_url)
except HTTPError as e:
    print(str(e).strip())
    file_path = '../data/html/minecraft_server_list.html'
    page_soup = get_page_soup(file_path)

HTTP Error 403: Forbidden


In [3]:

page_soup.select('a')

[<a href="//minecraft-server-list.com/server/7038/" title="EcoCityCraft Economy"><img alt="EcoCityCraft Economy" height="90" src="//cdn.minecraft-server-list.com/serverlogo/7038.jpg" width="680"/></a>,
 <a href="//minecraft-server-list.com/server/7038/" title="EcoCityCraft Economy Minecraft Server">EcoCityCraft Economy</a>,
 <a href="//minecraft-server-list.com/server/380867/" title="OPBlocks"><img alt="OPBlocks" height="90" src="//cdn.minecraft-server-list.com/serverlogo/380867.jpg" width="680"/></a>,
 <a href="//minecraft-server-list.com/server/380867/" title="OPBlocks Minecraft Server">OPBlocks</a>,
 <a href="//minecraft-server-list.com/server/259891/" title="GRM Pixelmon"><img alt="GRM Pixelmon" height="90" src="//cdn.minecraft-server-list.com/serverlogo/259891.jpg" width="680"/></a>,
 <a href="//minecraft-server-list.com/server/259891/" title="GRM Pixelmon Minecraft Server">GRM Pixelmon</a>,
 <a href="//minecraft-server-list.com/server/250183/" title="PURPLE PRISON"><img alt="PURP